In [46]:
import os
import glob
import numpy as np
import shutil

In [49]:
files = glob.glob("test/*")
files

['test/gettyimages-81888267-612x612.jpg',
 'test/gettyimages-1281554848-612x612.jpg',
 'test/front-lawn-of-suburban-house-royalty-free-image-1658177220.jpg',
 'test/gettyimages-157168323-612x612.jpg',
 'test/gettyimages-200478943-001-612x612.jpg',
 'test/istockphoto-157376658-612x612.jpg',
 'test/gettyimages-128502214-612x612.jpg',
 'test/gettyimages-157649292-612x612.jpg',
 'test/johnson-johnson-U6Q6zVDgmSs-unsplash.jpg',
 'test/gettyimages-76038132-612x612.jpg',
 'test/gettyimages-981616638-612x612.jpg',
 'test/gettyimages-1327080125-612x612.jpg',
 'test/gettyimages-171246403-612x612.jpg',
 'test/wallpaperflare.com_wallpaper.jpg',
 'test/phil-hearing-IYfp2Ixe9nM-unsplash.jpg',
 'test/gettyimages-184859699-612x612.jpg',
 'test/gettyimages-528689816-612x612.jpg',
 'test/gettyimages-1354034882-612x612.jpg',
 'test/jacques-bopp-Hh18POSx5qk-unsplash.jpg',
 'test/gettyimages-168338782-612x612.jpg',
 'test/gettyimages-76185214-612x612.jpg',
 'test/gettyimages-AA014332-612x612.jpg',
 'test/g

In [51]:
for i,file in enumerate(files):
    shutil.copy(file,f"images/{i}.jpg")
    txt = file.split("/")[-1].replace('.jpg','.txt')
    shutil.copy(f"ground_truth/{txt}",f"gnd_truths/{i}_gt.txt")

In [10]:
# 0 = door
# 1 = garrage
# 2 = window
import os
from ultralytics import YOLO

model = YOLO('weights/detetor/best_house.pt')  # load a custom model
files = os.listdir('yolov8_data/resized/images')
# Predict with the model
for file in files:
    results = model(f"yolov8_data/resized/images/{file}")
    boxes = results[0].boxes.xywhn.cpu().tolist()
    classes = [int(x) for x in results[0].boxes.cls.cpu().tolist()]
    confidence = results[0].boxes.conf.cpu().tolist()
    lines = []
    classes=[2 if x == 0 else 0 if x == 1 else 1 for x in classes]
    for i, box in enumerate(boxes):
        box.insert(0,confidence[i])
        box.insert(0, classes[i])
        lines.append(box)
    ann = file.split(".")[0]
    with open(f'yolov8_data/resized/pred/{ann}_pred.txt','w') as f:
        for line in lines:
            row_str = ' '.join(str(element) for element in line)  # Convert each element to string
            f.write(row_str + '\n')
    # break


image 1/1 /home/adlytic/Desktop/usama/SAM/yolov8_data/resized/images/7.jpg: 384x640 2 windows, 1 door, 1 garage, 3.6ms
Speed: 1.0ms preprocess, 3.6ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/adlytic/Desktop/usama/SAM/yolov8_data/resized/images/0.jpg: 384x640 4 windows, 1 door, 1 garage, 3.2ms
Speed: 0.9ms preprocess, 3.2ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/adlytic/Desktop/usama/SAM/yolov8_data/resized/images/31.jpg: 384x640 18 windows, 1 door, 3.8ms
Speed: 1.1ms preprocess, 3.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/adlytic/Desktop/usama/SAM/yolov8_data/resized/images/39.jpg: 384x640 12 windows, 3.7ms
Speed: 1.4ms preprocess, 3.7ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/adlytic/Desktop/usama/SAM/yolov8_data/resized/images/26.jpg: 384x640 7 windows, 1 door, 4.8ms
Speed: 1.4ms preprocess, 4.8ms inference, 1.0ms p

In [6]:
confs = results[0].boxes.conf.cpu().tolist()
boxes = results[0].boxes.xywhn.cpu().tolist()
classes

[0, 2, 1, 0]

In [9]:
classes

[2, 1, 0, 2]

In [94]:
lines = []
for i, box in enumerate(boxes):
    box.insert(0,confs[i])
    box.insert(0, classes[i])
    lines.append(box)
    break

# with open('test.txt','w') as f:
#     for line in lines:
#         row_str = ' '.join(str(element) for element in line)  # Convert each element to string
#         f.write(row_str + '\n')



In [95]:

def calculate_iou(gt_mask, pred_mask):
    intersection = np.logical_and(gt_mask, pred_mask)
    union = np.logical_or(gt_mask, pred_mask)
    iou = np.sum(intersection) / np.sum(union)
    return iou

def calculate_miou(gt_masks, pred_masks, num_classes):
    miou_sum = 0.0
    for class_id in range(num_classes):
        gt_class_mask = (gt_masks == class_id).astype(np.uint8)
        pred_class_mask = (pred_masks == class_id).astype(np.uint8)
        iou = calculate_iou(gt_class_mask, pred_class_mask)
        miou_sum += iou
    miou = miou_sum / num_classes
    return miou

# Example usage
gt_masks = np.array([   # Ground truth masks (shape: [num_samples, height, width])
    [[1, 0, 0], [0, 1, 1], [1, 1, 0]],
    [[1, 0, 1], [0, 1, 0], [0, 0, 1]]
])

pred_masks = np.array([   # Predicted masks (shape: [num_samples, height, width])
    [[1, 0, 0], [0, 1, 0], [1, 1, 0]],
    [[0, 0, 1], [1, 1, 0], [0, 0, 1]]
])

num_classes = 3

miou = calculate_miou(gt_masks, pred_masks, num_classes)
print("mIoU:", miou)

[0,
 0.9790241718292236,
 0.25459611415863037,
 0.2861822545528412,
 0.08900390565395355,
 0.14056110382080078]

In [62]:
boxes[-2]

[0.15911433100700378,
 0.5904607772827148,
 0.2113124132156372,
 0.6883756518363953]

In [4]:
import shutil
images = os.listdir('yolov8_data/resized/train/images/')
for i,img in enumerate(images):
    src_img = f"yolov8_data/resized/train/images/{img}"
    dest_img = f"yolov8_data/resized/images/{i}.jpg"
    ann = img.replace('.jpg','.txt')
    src_ann = f"yolov8_data/resized/train/labels/{ann}"
    dest_ann = f"yolov8_data/resized/anns/{i}_gt.txt"
    print(src_img,src_ann)
    print(dest_img,dest_ann)
    shutil.copy(src_img,dest_img)
    shutil.copy(src_ann,dest_ann)


yolov8_data/resized/train/images/gettyimages-175422480-612x612_jpg.rf.2aa7461eecbe3226967efd2264d46916.jpg yolov8_data/resized/train/labels/gettyimages-175422480-612x612_jpg.rf.2aa7461eecbe3226967efd2264d46916.txt
yolov8_data/resized/images/0.jpg yolov8_data/resized/anns/0_gt.txt
yolov8_data/resized/train/images/gettyimages-171246403-612x612_jpg.rf.3c952fc7d5873e342180440d60a6de50.jpg yolov8_data/resized/train/labels/gettyimages-171246403-612x612_jpg.rf.3c952fc7d5873e342180440d60a6de50.txt
yolov8_data/resized/images/1.jpg yolov8_data/resized/anns/1_gt.txt
yolov8_data/resized/train/images/gettyimages-1167399452-612x612_jpg.rf.4cb1e4c26d80b7f969108566dcaabae7.jpg yolov8_data/resized/train/labels/gettyimages-1167399452-612x612_jpg.rf.4cb1e4c26d80b7f969108566dcaabae7.txt
yolov8_data/resized/images/2.jpg yolov8_data/resized/anns/2_gt.txt
yolov8_data/resized/train/images/gettyimages-185269607-612x612_jpg.rf.8d0d0294f25483b7b517bb2403ecbb02.jpg yolov8_data/resized/train/labels/gettyimages-185